In [11]:
from matplotlib import image

In [12]:
image.imread('../../../../GWAS/TabFig/SNP_3D_FACE_DCOR/best_face_3d_dcorrs12637297_trip.pdf',format="pdf")

OSError: cannot identify image file '../../../../GWAS/TabFig/SNP_3D_FACE_DCOR/best_face_3d_dcorrs12637297_trip.pdf'

In [13]:
list(range(6))

[0, 1, 2, 3, 4, 5]

In [14]:
partial

NameError: name 'partial' is not defined

In [ ]:
import torch

In [ ]:
torch.stack([torch.from_numpy(b) for b in [0,1,1,0]],0)

In [1]:
# -*- coding: utf-8 -*
# author ChangeBio

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torchvision
from torch.optim import Adam
from torch.autograd import Variable

#from flyai.dataset import Dataset
from flyai.source.base import DATA_PATH

from model import Model
from net import Net, LSTMNet,FCNet
from path import MODEL_PATH
from processor import NLPFlyAI,load_data, make_weights_for_balanced_classes
from utils import Bunch

def eval(model, x_test, y_test):
    network.eval()
    total_acc = 0.0
    data_len = len(x_test[0])
    x1, x2 = x_test
    x1 = torch.from_numpy(x1)
    x2 = torch.from_numpy(x2)
    x1 = x1.float().to(device)
    x2 = x2.float().to(device)
    y_test = torch.from_numpy(y_test)
    y_test = y_test.to(device)
    batch_eval = model.batch_iter(x1, x2, y_test)

    for x_batch1, x_batch2, y_batch in batch_eval:
        outputs = network(x_batch1, x_batch2)
        _, prediction = torch.max(outputs.data, 1)
        correct = (prediction == y_batch).sum().item()
        total_acc += correct
    return total_acc / data_len

parser = argparse.ArgumentParser()
parser.add_argument("-e", "--EPOCHS", default=10, type=int, help="train epochs")
parser.add_argument("-b", "--BATCH", default=16, type=int, help="batch size")
args = parser.parse_args([])

#settings
settings = {
'net':'lstm',
'nc':2,    
'lr': 0.001,
'seed': 1,
'log_interval': 100,
'save_model': True,
'predict': True}
print("1.settings",settings)
settings = Bunch(settings)
use_cuda = torch.cuda.is_available()
torch.manual_seed(settings.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}

1.settings {'net': 'lstm', 'nc': 2, 'lr': 0.001, 'seed': 1, 'log_interval': 100, 'save_model': True, 'predict': True}


In [7]:
#load data
#data = Dataset()
#model = Model(data)
trn,val = load_data()
train_dataset = NLPFlyAI(root=DATA_PATH,df=trn)
val_dataset = NLPFlyAI(root=DATA_PATH,df=val)
weight = make_weights_for_balanced_classes(trn.label,settings.nc)
sampler = torch.utils.data.sampler.WeightedRandomSampler(weight,len(weight))
train_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=args.BATCH,sampler = sampler, **kwargs)
test_loader = torch.utils.data.DataLoader(val_dataset,
    batch_size=args.BATCH, **kwargs)
n_train = len(train_dataset)
batch_train = n_train/args.BATCH
print(args.BATCH,n_train/args.BATCH)
n_test = len(val_dataset)
batch_test = n_test/args.BATCH
print("2. load data. train_dataset %d,batch %d, val_dataset %d, batch %d." % (n_train,batch_train,n_test,batch_test))

{'id': 'SST', 'type': 'csv', 'config': {'train_url': 'https://dataset.flyai.com/dataset/SST/dev.zip', 'test_url': 'https://dataset.flyai.com/dataset/SST/dev.zip'}}
[200.0, 0.65, 0.47816651501375756, 0.0, 0.0, 1.0, 1.0, 1.0]
1    130
0     70
Name: label, dtype: int64
16 12.5
2. load data. train_dataset 200,batch 12, val_dataset 100, batch 6.


In [222]:
trn.iloc[50:60,]

,sentence,label
50,good old-fashioned adventure,1
51,exudes the fizz of a busby berkeley musical an...,1
52,the essence of magic,1
53,remarkable and,1
54,has transformed the rhetoric of hollywood melo...,1
55,the film acknowledges upfront that the plot ma...,0
56,is still charming,1
57,goofiest,1
58,with a taste for exaggeration,0
59,has made a film so unabashedly hopeful that it...,1


In [242]:
def words2vec(words,vocab,embedding_len,max_sts_len):
    words = str(words)
    words = re.sub("[\s+\.\!\/_,$%^*()+-?\"\']+|[+——！，。；？、~@#￥%……&*（）]+", " ", words)
    words = words.strip().split(' ')
    
    vecs = []
    for word in words:
        embedding_vector = vocab.get(word)
        if embedding_vector is not None:
                vecs.append(embedding_vector)
    if len(vecs) >= max_sts_len:
        vecs = vecs[:max_sts_len]
    elif embedding_len>1:
        for i in range(len(vecs), max_sts_len):
            vecs.append([0.0 for j in range(embedding_len)])
    else:
        for i in range(len(vecs), max_sts_len):
            vecs.append(0)
    vecs = np.stack(vecs)
    return vecs

In [229]:
import os
import json
import numpy as np

In [233]:
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import sys
import json

import jieba
import create_dict
import numpy as np
import os
import re
from flyai.processor.base import Base
from math import isnan

from time import time
import platform
import random
import requests
import pandas as pd
import numpy
import codecs
from flyai.processor.download import check_download
from flyai.utils.yaml_helper import Yaml
from flyai.utils import read_data

from path import DATA_PATH  # 导入输入数据的地址


In [230]:
embedding_path = os.path.join(DATA_PATH, 'embedding.json')
with open(embedding_path) as f:
    ch_vecs = json.loads(f.read())

new_ch_vecs = dict()
for k,v in ch_vecs.items():
    new_ch_vecs[k] = np.array([float(i) for i in v])

In [235]:
new_ch_vecs

{'gyllenhaal': array([-0.081241, -0.454638,  0.563093,  0.204763,  0.913456, -0.148959,
         0.794016,  0.337056,  0.332838,  0.296958,  0.054448, -0.02928 ,
         0.807909,  0.699976, -0.470027, -0.025245,  0.485276,  0.096838,
        -0.056124, -0.114371, -0.119038, -0.137179,  0.090544, -0.020224,
        -0.334027, -0.371958, -0.379486, -0.14234 ,  0.146407,  0.212891,
        -0.597451,  0.068407,  0.052209,  0.680058,  0.111148, -0.22952 ,
         0.636195, -0.551946,  0.204332,  0.135215, -0.026719,  0.406123,
         0.568153, -0.046294,  0.197636,  0.133883, -0.472154, -0.915403,
        -0.212469, -0.055921, -0.380773,  0.29519 , -0.414104,  0.521772,
         0.21585 , -0.079892,  0.09753 , -0.031213, -0.066756,  0.295859,
         0.296805,  0.423451,  0.9841  ,  0.217078, -0.09731 , -0.418614,
         0.036758, -0.001813,  0.413679,  0.242127, -0.351379,  0.117673,
        -0.359051,  0.530097,  0.026943,  0.516148,  0.14306 , -0.06687 ,
         0.268932,  0.41

In [237]:
trn.iloc[50,0]

'good old-fashioned adventure '

In [243]:
temp = [words2vec(trn.iloc[i,0],new_ch_vecs,200,20) for i in range(50,60)]

In [244]:
temp

[array([[-0.097308, -0.249725,  0.08001 , ..., -0.586716, -0.135169,
          0.207932],
        [-0.009415,  0.049524,  0.377373, ..., -0.741675, -0.313822,
          0.209074],
        [ 0.021127, -0.070912,  0.410513, ..., -0.343027, -0.279386,
          0.653862],
        ...,
        [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
          0.      ],
        [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
          0.      ],
        [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
          0.      ]]),
 array([[-0.232577, -0.027077,  0.177346, ..., -0.887316, -0.505458,
          0.155569],
        [ 0.046761, -0.157697,  0.103506, ..., -0.503052, -0.207889,
          0.309254],
        [-0.024769, -0.244185,  0.038844, ..., -0.630476, -0.233946,
          0.63393 ],
        ...,
        [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
          0.      ],
        [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
   

In [240]:
[i.shape for i in temp]

[(20, 200),
 (20, 200),
 (20, 200),
 (20, 200),
 (20, 200),
 (20, 200),
 (20, 200),
 (20, 200),
 (20, 200),
 (20, 200)]

In [180]:
list(BatchSampler(sampler, batch_size=args.BATCH, drop_last=False))

[[103, 17, 55, 156, 1, 25, 98, 117, 186, 9, 113, 156, 65, 146, 56, 44]]

In [28]:
from torch.utils.data.sampler import WeightedRandomSampler,BatchSampler

In [18]:
list(sampler)

[30, 139, 51, 117, 97, 14, 183, 12, 129, 167, 32, 150, 187, 64, 106, 159]

In [103]:
trn.iloc[list(sampler)]

,sentence,label
40,have an original bone in his body,0
32,"with its exquisite acting , inventive screenpl...",1
42,sneaks up on you and,1
21,has never been more charming than in about a b...,1
85,eyes,1
65,nutty cliches and far too much dialogue,0
2,be even worse than its title,0
26,a few nonbelievers may rethink their attitudes...,1
31,adam sandler 's eight crazy nights grows on yo...,0
25,spy kids 2 also happens to be that rarity amon...,1


In [124]:
for i in range(200):
    print(type(trn.iloc[0,1]))

<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'nu

In [161]:
next(iter(train_loader))

1 <class 'numpy.int64'>
1 <class 'numpy.int64'>
1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
1 <class 'numpy.int64'>
1 <class 'numpy.int64'>


[tensor([[[-0.0473, -0.2880, -0.1169,  ..., -0.6919, -0.3141,  0.3006],
          [ 0.1320, -0.7738, -0.0836,  ..., -0.3198, -0.4703,  0.1239],
          [ 0.0685,  0.0286,  0.2655,  ..., -0.5464,  0.0335,  0.2792],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[ 0.0945, -0.0177,  0.1960,  ..., -0.5760,  0.0669,  0.1538],
          [-0.2237, -0.2852,  0.0864,  ..., -0.3474, -0.1919,  0.2437],
          [ 0.1673, -0.2784,  0.1664,  ..., -0.3939, -0.1214,  0.2354],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[ 0.3925,  0.0934,  0.0017,  ..., -0.3450, -0.1866,  0.5867],
          [ 0.0490, -0.0846,

In [162]:
torch.stack(torch.from_numpy([1,2,3]),0)

TypeError: expected np.ndarray (got list)

In [15]:
torch.Tensor([0])

tensor([0.])

In [6]:
type(trn.iloc[0,1])

numpy.int64

In [8]:
trn.iloc[0,1]==1

True

In [98]:
list(iter(train_loader))

1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
1 <class 'numpy.int64'>
1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
1 <class 'numpy.int64'>
1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
0 <class 'numpy.int64'>
1 <class 'numpy.int64'>
0 <class 'numpy.int64'>
1 <class 'numpy.int64'>
1 <class 'numpy.int64'>
1 <class 'numpy.int64'>


RuntimeError: Expected object of scalar type Double but got scalar type Long for sequence elment 7 in sequence argument at position #1 'tensors'

In [ ]:
torch.stack([i for i in trn.label])

In [9]:
#load net structure
print("3.load net structure: %s, number of class: %d" % (settings.net,settings.nc))
net = LSTMNet()
gpu = torch.cuda.is_available()
if gpu:
    net.cuda()
#optimize and loss
print("4.optimize and loss. learning rate %g" % settings.lr)
optimizer = Adam(net.parameters(), lr=settings.lr, weight_decay=1e-4)
#optimizer = Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999))  
loss_fn = nn.CrossEntropyLoss()  

#train and test
print("5.***************train and test*********************")
best_accuracy = 0
train_iter = iter(train_loader)
batch_idx = 0
for epoch in range(args.EPOCHS):
    net.train()
    #x_train, y_train, x_test, y_test = data.next_batch(args.BATCH)  # 读取数据
    #batch_len = y_train.shape[0]
    #x1, x2 = x_train
    #x1 = torch.from_numpy(x1)
    #x2 = torch.from_numpy(x2)
    #x1 = x1.float().to(device)
    #x2 = x2.float().to(device)
    #y_train = torch.from_numpy(y_train)
    #y_train = y_train.to(device)
    try:
        batch_idx +=1
        x1, y_train = next(train_iter)
        print(batch_idx,"data len",x1.shape,len(y_train))
    except:
        batch_idx = 0
        train_iter = iter(train_loader)
        x1, y_train = next(train_iter)
        print(len(x1),len(y_train),"data len")
        print(batch_idx,"data len",x1.shape,len(y_train))

    if gpu:
        x1 = Variable(x1.cuda())
        #x2 = Variable(x2.cuda())
        #x3 = Variable(x3.cuda())
        y_train = Variable(y_train).type(torch.LongTensor)
        y_train = y_train.cuda()


    outputs = net(x1)
    optimizer.zero_grad()
    loss = loss_fn(outputs, y_train)
    loss.backward()
    optimizer.step()
    if batch_idx % settings.log_interval == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(y_train), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

if settings.save_model:       
    model.save_model(net, MODEL_PATH, overwrite=True)

if settings.predict:
    print(x1,y_train)
    import random
    _,_,x_test,y_text = data.get_all_data()
    test_idx = random.sample(range(len(x_test)),10)
    print(model.predict_all([x_test[i] for i in test_idx]),[y_text[i] for i in test_idx])

3.load net structure: lstm, number of class: 2
4.optimize and loss. learning rate 0.001
5.***************train and test*********************
1 data len torch.Size([16, 20, 200]) 16
2 data len torch.Size([16, 20, 200]) 16
3 data len torch.Size([16, 20, 200]) 16
4 data len torch.Size([16, 20, 200]) 16
5 data len torch.Size([16, 20, 200]) 16
6 data len torch.Size([16, 20, 200]) 16
7 data len torch.Size([16, 20, 200]) 16
8 data len torch.Size([16, 20, 200]) 16
9 data len torch.Size([16, 20, 200]) 16
10 data len torch.Size([16, 20, 200]) 16


NameError: name 'model' is not defined

In [8]:
len(list(sampler))

200

In [10]:
from torch.utils.data.sampler import WeightedRandomSampler

In [16]:
list(WeightedRandomSampler([0.1, 0.9, 0.4, 0.7, 3.0, 0.6], 5, replacement=True))

[1, 4, 1, 4, 5]

In [ ]:
x1, y_train = next(iter(train_loader))

In [ ]:
y_train

In [ ]:
x1

In [ ]:
temp=[train_dataset[i] for i in range(5)]

In [ ]:
temp

In [ ]:
temp[0]

In [ ]:
hasattr(temp[0], '_fields')